In [1]:
library(dplyr)
library(readr)   # read csv file
library(texreg)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Version:  1.37.5
Date:     2020-06-17
Author:   Philip Leifeld (University of Essex)

Consider submitting praise using the praise or praise_interactive functions.
Please cite the JSS article in your publications -- see citation("texreg").


### <div class='info-circle alert alert-block alert-info'> Load combined data</div>

In [2]:
combined_df <- read_csv('Data/combined_data.csv')
dim(combined_df)

Parsed with column specification:
cols(
  .default = col_double(),
  city_cn = col_character(),
  city = col_character(),
  province_cn = col_character(),
  province = col_character(),
  tier = col_character()
)
See spec(...) for full column specifications.


[1] 366  97

In [3]:
head(combined_df)

city_cn,city,city_id,province_cn,province,tier,lat,lng,population,GDP,...,date_0324,date_0325,date_0326,date_0327,date_0328,date_0329,date_0330,date_0331,date_0401,hubei_city
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
北京市,Beijing,110000,北京市,Beijing,Tier 1,39.91092,116.4134,2153.60,35371.30,...,559,565,569,572,576,577,580,580,582,0
天津市,Tianjin,120000,天津市,Tianjin,new Tier 1,39.09367,117.2095,1561.83,14104.00,...,145,147,152,156,163,166,174,174,176,0
石家庄市,Shijiazhuang,130100,河北省,Hebei,Tier 2,38.04831,114.5215,1039.42,5809.90,...,29,29,29,29,29,29,29,29,29,0
唐山市,Tangshan,130200,河北省,Hebei,Tier 3,39.63658,118.1865,796.40,6890.00,...,58,58,58,58,58,58,58,58,58,0
秦皇岛市,Qinhuangdao,130300,河北省,Hebei,Tier 3,39.94175,119.6085,314.63,1612.02,...,10,10,10,10,10,10,10,10,10,0
邯郸市,Handan,130400,河北省,Hebei,Tier 3,36.63126,114.5456,954.97,3486.00,...,32,32,32,32,32,32,32,32,32,0


In [4]:
table(combined_df$hubei_city)


  0   1   2 
349  16   1 

### <div class="alert alert-block alert-success"> Regression Analysis </div>
### Economic development

GDP_growth_rate ~ log_GDP_pc + GDP_growth_2019 + log_cumulative_cases + city_in_hubei + intensity_2020 + intensity_reduction


In [5]:
reg_df <- combined_df

# cases on '2020-02-09'
reg_df$cumulative_cases <- reg_df$date_0209

reg_df <- reg_df %>% 
                mutate(city_in_hubei = ifelse(hubei_city==0, 0, 1)) %>%
                mutate(log_GDP_pc = log(GDP_pc)) %>%
                mutate(log_cumulative_cases = log(cumulative_cases+1))

In [6]:
ols_model <- lm(GDP_growth_rate ~ 
                    log_GDP_pc + 
                    GDP_growth_2019 + 
                    log_cumulative_cases + 
                    city_in_hubei +
                    intensity_2020 +
                    intensity_reduction, 
              data = reg_df
                   )

summary(ols_model)


Call:
lm(formula = GDP_growth_rate ~ log_GDP_pc + GDP_growth_2019 + 
    log_cumulative_cases + city_in_hubei + intensity_2020 + intensity_reduction, 
    data = reg_df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.38150 -0.02929  0.00141  0.03462  0.20022 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           0.6365380  0.0400072  15.911  < 2e-16 ***
log_GDP_pc            0.0396089  0.0079764   4.966 1.11e-06 ***
GDP_growth_2019       0.0086114  0.0020097   4.285 2.42e-05 ***
log_cumulative_cases  0.0008497  0.0030738   0.276  0.78239    
city_in_hubei        -0.2509303  0.0234751 -10.689  < 2e-16 ***
intensity_2020        0.0003268  0.0001057   3.093  0.00216 ** 
intensity_reduction  -0.0002981  0.0001301  -2.292  0.02255 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.06801 on 323 degrees of freedom
  (36 observations deleted due to missingness)
Multiple R-squared:  0.4

#### Hierarchical OLS regression on economic development

In [7]:
reduced_model <- lm(GDP_growth_rate ~ log_GDP_pc + GDP_growth_2019 + log_cumulative_cases +
                            intensity_2020+intensity_reduction,
              data = reg_df
            )

In [8]:
summary(reduced_model)


Call:
lm(formula = GDP_growth_rate ~ log_GDP_pc + GDP_growth_2019 + 
    log_cumulative_cases + intensity_2020 + intensity_reduction, 
    data = reg_df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.38166 -0.03941  0.00716  0.04933  0.23545 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           0.6191739  0.0464383  13.333  < 2e-16 ***
log_GDP_pc            0.0545220  0.0091234   5.976 6.02e-09 ***
GDP_growth_2019       0.0064423  0.0023228   2.774  0.00587 ** 
log_cumulative_cases -0.0141815  0.0031752  -4.466 1.10e-05 ***
intensity_2020        0.0004948  0.0001214   4.076 5.78e-05 ***
intensity_reduction  -0.0004430  0.0001503  -2.948  0.00344 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.07901 on 324 degrees of freedom
  (36 observations deleted due to missingness)
Multiple R-squared:  0.2782,	Adjusted R-squared:  0.267 
F-statistic: 24.97 on 5 and 324 DF,  p-value: <

In [9]:
print(texreg(list(reduced_model, ols_model), digits=4))


\begin{table}
\begin{center}
\begin{tabular}{l c c}
\hline
 & Model 1 & Model 2 \\
\hline
(Intercept)            & $0.6192^{***}$  & $0.6365^{***}$  \\
                       & $(0.0464)$      & $(0.0400)$      \\
log\_GDP\_pc           & $0.0545^{***}$  & $0.0396^{***}$  \\
                       & $(0.0091)$      & $(0.0080)$      \\
GDP\_growth\_2019      & $0.0064^{**}$   & $0.0086^{***}$  \\
                       & $(0.0023)$      & $(0.0020)$      \\
log\_cumulative\_cases & $-0.0142^{***}$ & $0.0008$        \\
                       & $(0.0032)$      & $(0.0031)$      \\
intensity\_2020        & $0.0005^{***}$  & $0.0003^{**}$   \\
                       & $(0.0001)$      & $(0.0001)$      \\
intensity\_reduction   & $-0.0004^{**}$  & $-0.0003^{*}$   \\
                       & $(0.0002)$      & $(0.0001)$      \\
city\_in\_hubei        &                 & $-0.2509^{***}$ \\
                       &                 & $(0.0235)$      \\
\hline
R$^2$                  & $0.2782$ 